In [1]:
import numpy as np
import filecmp
import pynq.lib.dma
from pynq import Xlnk
import time
from pynq import Overlay
from pynq import MMIO
ol= Overlay("kociembaMultipleDMA.bit")

maxDepthBaseAddress = 0x080
unsolvableBaseAddress = 0x088
coloursBaseAddress = 0x040
orderBaseAddress = 0x090
moveArrayBaseAddress = 0x0a0
mmio=MMIO(0x43C00000,0x10000)

mmio.write(maxDepthBaseAddress, 30)
print(mmio.read(maxDepthBaseAddress))

def readfile_short(fileName):
    f = open(fileName,'rb')
    buffer= f.read()
    lSize=f.tell()
    buffer_short = np.fromstring(buffer,dtype=np.short)
    open(cmpFile, 'w').close()
    f_out = open(cmpFile,'w+b')
    f_out.write(buffer_short)
    f.close()
    f_out.close()
    return buffer_short, int(lSize/2)

def readfile_int(fileName):
    f = open(fileName,'rb')
    buffer= f.read()
    lSize=f.tell()
    buffer_int = np.fromstring(buffer,dtype=np.intc)
    open(cmpFile, 'w').close()
    f_out = open(cmpFile,'w+b')
    f_out.write(buffer_short)
    f.close()
    f_out.close()
    return buffer_short, int(lSize/2)

def readfile_char(fileName):
    f = open(fileName,'rb')
    buffer= f.read()
    lSize=f.tell()
    buffer_char = np.fromstring(buffer,dtype=np.byte)
    open(cmpFile, 'w').close()
    f_out = open(cmpFile,'w+b')
    f_out.write(buffer_char)
    f.close()
    f_out.close()
    return buffer_char, lSize
#######################################################################################################################
xlnk = Xlnk()
xlnk.xlnk_reset()

cmpFile = "results/cmp"

list_file_short = ["results/twistMove",\
                   "results/flipMove",\
                   "results/FRtoBR_Move",\
                   "results/URFtoDLF_Move",\
                   "results/URtoDF_Move",\
                   "results/URtoUL_Move",\
                   "results/UBtoDF_Move",\
                   "results/MergeURtoULandUBtoDF"]

list_file_char = ["results/Slice_URFtoDLF_Parity_Prun",\
                  "results/Slice_URtoDF_Parity_Prun",\
                  "results/Slice_Twist_Prun",\
                  "results/Slice_Flip_Prun"]

pruneTableMem=[]

###############################################################################################################################
j =0;
for file in list_file_short:
    pruneTable, size = readfile_short(file)
    if (filecmp.cmp(file,cmpFile)==False):
        print("file compared fail")
    pruneTableMem.append(xlnk.cma_array(shape=(size,), dtype=np.short))
    for i in np.arange(size):
        pruneTableMem[j][i]=pruneTable[i]
    j = j + 1
        
for file in list_file_char:
    pruneTable, size = readfile_char(file)
    if (filecmp.cmp(file,cmpFile)==False):
        print("file compared fail")
    pruneTableMem.append(xlnk.cma_array(shape=(size,), dtype=np.byte))
    for i in np.arange(size):
        pruneTableMem[j][i]=pruneTable[i]
    j = j + 1
    
##############################################################################################################################
pruneTable_address=[0x010,0x018,0x020,0x028,0x030,0x038,0x098,0x0c0,0x0c8,0x0d0,0x0d8,0x0e0 ]

for i in np.arange(len(pruneTable_address)):
    mmio.write(pruneTable_address[i],pruneTableMem[i].physical_address)
    

30


In [2]:
moves= []

def assign(mNum):
    if(mNum==0):
        moves.append("U")
    elif(mNum==1):
        moves.append("R")
    elif(mNum==2):
        moves.append("F")
    elif(mNum==3):
        moves.append("D")
    elif(mNum==4):
        moves.append("L")
    elif(mNum==5):
        moves.append("B")
    elif(mNum==6):
        moves.append("U'")
    elif(mNum==7):
        moves.append("R'")
    elif(mNum==8):
        moves.append("F'")
    elif(mNum==9):
        moves.append("D'")
    elif(mNum==10):
        moves.append("L'")
    elif(mNum==11):
        moves.append("B'")
    elif(mNum==12):
        moves.append("U2")
    elif(mNum==13):
        moves.append("R2")
    elif(mNum==14):
        moves.append("F2")
    elif(mNum==15):
        moves.append("D2")
    elif(mNum==16):
        moves.append("L2")
    elif(mNum==17):
        moves.append("B2")
    else:
        print("error in coverting value in memory to move")

def bytes(num):
    assign(num& 0xff) 
    assign(num >>8 & 0xff)
    assign(num >> 16 & 0xff)
    assign(num >> 24& 0xff)

def solve(colours):
    mmio.write(0x0a0,0)
    for i in np.arange(0,52,4):
        coloursMem = ord(colours[i]) + (ord(colours[i+1])<< 8) + (ord(colours[i+2])<< 16) +(ord(colours[i+3])<< 24)
        mmio.write(coloursBaseAddress+ i ,coloursMem)
    coloursMem= ord(colours[52]) +(ord(colours[53])<< 8)
    mmio.write(coloursBaseAddress+ 52 ,coloursMem)
    print(mmio.read(0x000))
    mmio.write(0x000, 1)
    start_time = time.time()
    while ((mmio.read(0x000)& 0x2)!=2):
#     while ((mmio.read(0x94))!=1):
#         print(hex(mmio.read(0x0a0)), end = ' ')
        pass
    elapsed_time = time.time() - start_time
    order= int(mmio.read(orderBaseAddress))
    j=0
    moves.clear()
    if (mmio.read(unsolvableBaseAddress)==1):
        print("Unsolvable")
        return
    while( j<order):
        data = mmio.read(int(moveArrayBaseAddress)+j)
        bytes(data)
        j=j+4
    print("Number of moves is ", order)
    print("elapsed time is ", elapsed_time)
    print(moves[0:order])

In [3]:
solve("DRLUUBFBRBLURRLRUBLRDDFDLFUFUFFDBRDUBRUFLLFDDBFLUBLRBD")
solve("FLBUULFFLFDURRDBUBUUDDFFBRDDBLRDRFLLRLRULFUDRRBDBBBUFL")
solve("BBURUDBFUFFFRRFUUFLULUFUDLRRDBBDBDBLUDDFLLRRBRLLLBRDDF")
solve("UUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBB")
solve("BBBBBBBBBUUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLL")
solve("LLLLLLLLLBBBBBBBBBUUUUUUUUURRRRRRRRRFFFFFFFFFDDDDDDDDD")
solve("DDDDDDDDDLLLLLLLLLBBBBBBBBBUUUUUUUUURRRRRRRRRFFFFFFFFF")
solve("FFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBBUUUUUUUUURRRRRRRRR")
solve("RRRRRRRRRFFFFFFFFFDDDDDDDDDLLLLLLLLLBBBBBBBBBUUUUUUUUU")

4
Number of moves is  19
elapsed time is  0.030233144760131836
["L'", 'B', "B'", "R'", 'D2', "D'", 'L', "L'", 'B', "R'", "B'", "R'", 'F', 'L2', 'R2', 'R', "D'", 'L', 'U']
4
Number of moves is  21
elapsed time is  1.0073015689849854
['R', "D'", 'B2', 'R', 'B', "L'", 'R2', "D'", 'L', 'U2', 'B2', 'F', 'R2', "D'", 'R2', 'U', 'R2', 'L2', 'L', 'R', "D'"]
4
Number of moves is  21
elapsed time is  1.5705459117889404
['D2', 'F', 'F2', "B'", 'B', "B'", 'R2', "B'", 'U2', "F'", 'F2', "D'", "R'", 'L', 'U', 'L', 'L2', 'R', 'R2', "R'", "B'"]
4
Number of moves is  0
elapsed time is  3.2901763916015625e-05
[]
4
Number of moves is  0
elapsed time is  3.218650817871094e-05
[]
4
Number of moves is  0
elapsed time is  3.266334533691406e-05
[]
4
Number of moves is  0
elapsed time is  3.24249267578125e-05
[]
4
Number of moves is  0
elapsed time is  3.170967102050781e-05
[]
4
Number of moves is  0
elapsed time is  3.361701965332031e-05
[]


In [6]:
for i in np.arange(20):
    solve("DRLUUBFBRBLURRLRUBLRDDFDLFUFUFFDBRDUBRUFLLFDDBFLUBLRBD")


4
Number of moves is  19
elapsed time is  0.028714418411254883
["L'", 'B', "B'", "R'", 'D2', "D'", 'L', "L'", 'B', "R'", "B'", "R'", 'F', 'L2', 'R2', 'R', "D'", 'L', 'U']
4
Number of moves is  19
elapsed time is  0.0008749961853027344
["L'", 'B', "B'", "R'", 'D2', "D'", 'L', "L'", 'B', "R'", "B'", "R'", 'F', 'L2', 'R2', 'R', "D'", 'L', 'U']
4
Number of moves is  19
elapsed time is  0.0008740425109863281
["L'", 'B', "B'", "R'", 'D2', "D'", 'L', "L'", 'B', "R'", "B'", "R'", 'F', 'L2', 'R2', 'R', "D'", 'L', 'U']
4
Number of moves is  19
elapsed time is  0.0008714199066162109
["L'", 'B', "B'", "R'", 'D2', "D'", 'L', "L'", 'B', "R'", "B'", "R'", 'F', 'L2', 'R2', 'R', "D'", 'L', 'U']
4
Number of moves is  19
elapsed time is  0.0008730888366699219
["L'", 'B', "B'", "R'", 'D2', "D'", 'L', "L'", 'B', "R'", "B'", "R'", 'F', 'L2', 'R2', 'R', "D'", 'L', 'U']
4
Number of moves is  19
elapsed time is  0.0008726119995117188
["L'", 'B', "B'", "R'", 'D2', "D'", 'L', "L'", 'B', "R'", "B'", "R'", 'F', 'L

In [4]:
for i in np.arange(20):
    solve("FLBUULFFLFDURRDBUBUUDDFFBRDDBLRDRFLLRLRULFUDRRBDBBBUFL")


4
Number of moves is  21
elapsed time is  1.0129756927490234
['R', "D'", 'B2', 'R', 'B', "L'", 'R2', "D'", 'L', 'U2', 'B2', 'F', 'R2', "D'", 'R2', 'U', 'R2', 'L2', 'L', 'R', "D'"]
4
Number of moves is  21
elapsed time is  0.006025791168212891
['R', "D'", 'B2', 'R', 'B', "L'", 'R2', "D'", 'L', 'U2', 'B2', 'F', 'R2', "D'", 'R2', 'U', 'R2', 'L2', 'L', 'R', "D'"]
4
Number of moves is  21
elapsed time is  0.006012916564941406
['R', "D'", 'B2', 'R', 'B', "L'", 'R2', "D'", 'L', 'U2', 'B2', 'F', 'R2', "D'", 'R2', 'U', 'R2', 'L2', 'L', 'R', "D'"]
4
Number of moves is  21
elapsed time is  0.006012439727783203
['R', "D'", 'B2', 'R', 'B', "L'", 'R2', "D'", 'L', 'U2', 'B2', 'F', 'R2', "D'", 'R2', 'U', 'R2', 'L2', 'L', 'R', "D'"]
4
Number of moves is  21
elapsed time is  0.006013154983520508
['R', "D'", 'B2', 'R', 'B', "L'", 'R2', "D'", 'L', 'U2', 'B2', 'F', 'R2', "D'", 'R2', 'U', 'R2', 'L2', 'L', 'R', "D'"]
4
Number of moves is  21
elapsed time is  0.00601506233215332
['R', "D'", 'B2', 'R', 'B', "L

In [5]:
for i in np.arange(20):
    solve("BBURUDBFUFFFRRFUUFLULUFUDLRRDBBDBDBLUDDFLLRRBRLLLBRDDF")

4
Number of moves is  21
elapsed time is  1.5708587169647217
['D2', 'F', 'F2', "B'", 'B', "B'", 'R2', "B'", 'U2', "F'", 'F2', "D'", "R'", 'L', 'U', 'L', 'L2', 'R', 'R2', "R'", "B'"]
4
Number of moves is  21
elapsed time is  0.004851579666137695
['D2', 'F', 'F2', "B'", 'B', "B'", 'R2', "B'", 'U2', "F'", 'F2', "D'", "R'", 'L', 'U', 'L', 'L2', 'R', 'R2', "R'", "B'"]
4
Number of moves is  21
elapsed time is  0.0048487186431884766
['D2', 'F', 'F2', "B'", 'B', "B'", 'R2', "B'", 'U2', "F'", 'F2', "D'", "R'", 'L', 'U', 'L', 'L2', 'R', 'R2', "R'", "B'"]
4
Number of moves is  21
elapsed time is  0.004853725433349609
['D2', 'F', 'F2', "B'", 'B', "B'", 'R2', "B'", 'U2', "F'", 'F2', "D'", "R'", 'L', 'U', 'L', 'L2', 'R', 'R2', "R'", "B'"]
4
Number of moves is  21
elapsed time is  0.004857063293457031
['D2', 'F', 'F2', "B'", 'B', "B'", 'R2', "B'", 'U2', "F'", 'F2', "D'", "R'", 'L', 'U', 'L', 'L2', 'R', 'R2', "R'", "B'"]
4
Number of moves is  21
elapsed time is  0.004845142364501953
['D2', 'F', 'F2', 

In [179]:
ol= Overlay("kociemba100M.bit")

mmio=MMIO(0x43C00000,0x10000)

In [175]:
mmio.read(0x000)

4

In [173]:
colours="BBURUDBFUFFFRRFUUFLULUFUDLRRDBBDBDBLUDDFLLRRBRLLLBRDDF"
for i in np.arange(0,52,4):
    coloursMem = ord(colours[i]) + (ord(colours[i+1])<< 8) + (ord(colours[i+2])<< 16) +(ord(colours[i+3])<< 24)
    mmio.write(coloursBaseAddress+ i ,coloursMem)
coloursMem= ord(colours[52]) +(ord(colours[53])<< 8)
mmio.write(coloursBaseAddress+ 52 ,coloursMem)

In [20]:
for i in np.arange(20):
    solve("FRRLUBUUBUDURRLLBBBLLUFBRRFDFUDDUDURDBRFLRLFFFDLDBLDFB")
    solve("DBDRULDUBLBFDRRULFFBDLFRUDFBFRBDFULURURULUBRRLDBFBFLDL")


4
Number of moves is  21
elapsed time is  2.332616090774536
['B', 'F', "R'", 'R2', 'U', 'D', 'L2', 'L', 'F2', 'F', "F'", 'U', "R'", "L'", 'L2', "B'", 'R2', 'F', "B'", "R'", 'L']
4
Number of moves is  20
elapsed time is  0.2530386447906494
['F2', "U'", 'L2', 'B', 'D2', 'B', 'U2', 'D2', "B'", "F'", 'U', 'L2', 'U', "R'", "B'", 'R2', 'L', 'U', "R'", "B'"]
4
Number of moves is  21
elapsed time is  2.3388113975524902
['B', 'F', "R'", 'R2', 'U', 'D', 'L2', 'L', 'F2', 'F', "F'", 'U', "R'", "L'", 'L2', "B'", 'R2', 'F', "B'", "R'", 'L']
4
Number of moves is  20
elapsed time is  0.25388145446777344
['F2', "U'", 'L2', 'B', 'D2', 'B', 'U2', 'D2', "B'", "F'", 'U', 'L2', 'U', "R'", "B'", 'R2', 'L', 'U', "R'", "B'"]
4
Number of moves is  21
elapsed time is  2.352130174636841
['B', 'F', "R'", 'R2', 'U', 'D', 'L2', 'L', 'F2', 'F', "F'", 'U', "R'", "L'", 'L2', "B'", 'R2', 'F', "B'", "R'", 'L']
4
Number of moves is  20
elapsed time is  0.25301551818847656
['F2', "U'", 'L2', 'B', 'D2', 'B', 'U2', 'D2', "B'